mounting to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


install library prerequisites

In [2]:
!pip install numpy scikit-learn pillow tqdm

script for finding the folders in drive

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [4]:
# Print the contents of the ASL_to_Text_Project directory
project_dir = '/content/drive/MyDrive/ASL_to_Text_Project'
print(f"\nContents of {project_dir}:")
print(os.listdir(project_dir))

# Print the contents of the data directory
data_dir = os.path.join(project_dir, 'data')
print(f"\nContents of {data_dir}:")
print(os.listdir(data_dir))


Contents of /content/drive/MyDrive/ASL_to_Text_Project:
['data', 'models']

Contents of /content/drive/MyDrive/ASL_to_Text_Project/data:
['labels', 'images']


training script

In [5]:
IMG_SIZE = 227
BATCH_SIZE = 32
EPOCHS = 100
DATA_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/data"
MODEL_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/models"
LABELS_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/data/labels"


# Configuration

def create_improved_model(num_classes):
    model = Sequential([
        Conv2D(64, 11, strides=4, padding="same", activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        BatchNormalization(),
        MaxPooling2D(3, strides=2),
        Conv2D(192, 5, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(3, strides=2),
        Conv2D(384, 3, padding="same", activation="relu"),
        BatchNormalization(),
        Conv2D(256, 3, padding="same", activation="relu"),
        BatchNormalization(),
        Conv2D(256, 3, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(3, strides=2),
        Flatten(),
        Dense(4096, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4096, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation="softmax")
    ])
    return model

def load_balanced_data(data_dir, img_size=IMG_SIZE, max_samples_per_class=1000):
    images = []
    labels = []
    class_counts = {}
    images_dir = os.path.join(data_dir, "images")

    for label in os.listdir(images_dir):
        label_dir = os.path.join(images_dir, label)
        if os.path.isdir(label_dir):
            class_counts[label] = 0
            for img_name in os.listdir(label_dir):
                if class_counts[label] >= max_samples_per_class:
                    break
                img_path = os.path.join(label_dir, img_name)
                if os.path.isfile(img_path):
                    try:
                        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
                        img_array = tf.keras.preprocessing.image.img_to_array(img)
                        images.append(img_array)
                        labels.append(label)
                        class_counts[label] += 1
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")

    print("Class distribution:")
    for label, count in class_counts.items():
        print(f"{label}: {count}")

    return np.array(images), np.array(labels)

def main():
    X, y = load_balanced_data(DATA_DIR)

    if len(X) == 0:
        print("No images were loaded. Please check the data directory structure.")
        return

    print(f"Loaded {len(X)} images with shape {X.shape}")
    print(f"Unique labels: {np.unique(y)}")

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    num_classes = len(le.classes_)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )

    train_generator = train_datagen.flow(
        X_train, y_train, batch_size=BATCH_SIZE, shuffle=True
    )

    model = create_improved_model(num_classes)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=15, restore_best_weights=True
    )
    lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=7)

    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, lr_scheduler]
    )

    test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nTest accuracy: {test_acc}")
    print(f"Test precision: {test_precision}")
    print(f"Test recall: {test_recall}")

    os.makedirs(MODEL_DIR, exist_ok=True)
    model_path = os.path.join(MODEL_DIR, "improved_balanced_asl_model.h5")
    model.save(model_path)
    print(f"Model saved successfully: {model_path}")

    os.makedirs(LABELS_DIR, exist_ok=True)
    label_encoder_path = os.path.join(LABELS_DIR, "label_encoder.pkl")
    with open(label_encoder_path, "wb") as f:
        pickle.dump(le, f)
    print(f"Label Encoder saved successfully: {label_encoder_path}")

if __name__ == "__main__":
    main()


Class distribution:
G: 500
F: 500
C: 500
H: 500
E: 500
I love you: 500
B: 500
A: 500
I: 500
D: 500
Q: 513
L: 500
M: 500
P: 500
N: 500
No: 500
Please: 500
O: 500
J: 500
K: 500
S: 500
V: 500
Thank you: 500
Unknown: 500
X: 500
U: 500
T: 500
W: 500
R: 500
Y: 500
Yes: 500
Z: 500
Loaded 16013 images with shape (16013, 227, 227, 3)
Unique labels: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'I love you' 'J' 'K' 'L' 'M' 'N' 'No'
 'O' 'P' 'Please' 'Q' 'R' 'S' 'T' 'Thank you' 'U' 'Unknown' 'V' 'W' 'X'
 'Y' 'Yes' 'Z']


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


400/400 ━━━━━━━━━━━━━━━━━━━━ 149s 336ms/step - accuracy: 0.2209 - loss: 3.4634 - precision: 0.3529 - recall: 0.1528 - val_accuracy: 0.0312 - val_loss: 10.0946 - val_precision: 0.0327 - val_recall: 0.0312 - learning_rate: 1.0000e-04
Epoch 2/100
  1/400 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7188 - loss: 1.0042 - precision: 0.7333 - recall: 0.6875

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7188 - loss: 1.0042 - precision: 0.7333 - recall: 0.6875 - val_accuracy: 0.0312 - val_loss: 10.0562 - val_precision: 0.0327 - val_recall: 0.0312 - learning_rate: 1.0000e-04
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 127s 313ms/step - accuracy: 0.7187 - loss: 0.8767 - precision: 0.7606 - recall: 0.6780 - val_accuracy: 0.9307 - val_loss: 0.1885 - val_precision: 0.9340 - val_recall: 0.9279 - learning_rate: 1.0000e-04
Epoch 4/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.8314 - precision: 0.9000 - recall: 0.8438 - val_accuracy: 0.9310 - val_loss: 0.1894 - val_precision: 0.9341 - val_recall: 0.9288 - learning_rate: 1.0000e-04
Epoch 5/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 127s 312ms/step - accuracy: 0.8883 - loss: 0.3488 - precision: 0.9002 - recall: 0.8772 - val_accuracy: 0.9241 - val_loss: 0.2513 - val_precision: 0.9247 - val_recall: 0.9235 - learning_rate: 1.0000e-04
Epoch 6/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms


Test accuracy: 1.0
Test precision: 1.0
Test recall: 1.0
Model saved successfully: /content/drive/MyDrive/ASL_to_Text_Project/models/improved_balanced_asl_model.h5
Label Encoder saved successfully: /content/drive/MyDrive/ASL_to_Text_Project/data/labels/label_encoder.pkl
